### Load necessary libraries

In [1]:
import numpy as np
import pandas as pd
import neuralsens.partial_derivatives as ns
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import set_config
set_config(display='diagram')
%matplotlib qt

### Create synthetic dataset to check behavior of functions

In [2]:
samples = 100000
n_columns = 8
sm = np.random.normal(size=(samples,n_columns))
df = pd.DataFrame(sm, columns=['X' + str(x) for x in range(1,n_columns+1)])

### Check behavior of Jacobian function

#### Create output Y as linear function of inputs with some non-linear relationship

In [3]:
df['Y'] = - 0.8 * df.X1 + 0.5 * df.X2 ** 2 - df.X3 * df.X4 + 0.1 * np.random.normal(size=(samples,)) 

#### Train MLP model using the data.frame created

In [4]:
## Create random 80/20 % split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Y'].to_numpy(), df['Y'], test_size = 0.2, random_state = 5)

In [5]:
### Create MLP model
model = MLPRegressor(solver='sgd', # Update function
                    hidden_layer_sizes=[10], # #neurons in hidden layers
                    learning_rate_init=0.001, # initial learning rate
                    activation='logistic', # Logistic sigmoid activation function
                    alpha=0.01, # L2 regularization term
                    learning_rate='adaptive', # Type of learning rate used in training
                    max_iter=500, # Maximum number of iterations
                    batch_size=10, # Size of batch when training
                    tol=1e-2, # Tolerance for the optimization
                    validation_fraction=0.0, # Percentage of samples used for validation
                    n_iter_no_change=10, # Maximum number of epochs to not meet tol improvement
                    random_state=150)

# Train model
model.fit(X_train, y_train)

MLPRegressor(activation='logistic', alpha=0.01, batch_size=10,
             hidden_layer_sizes=[10], learning_rate='adaptive', max_iter=500,
             random_state=150, solver='sgd', tol=0.01, validation_fraction=0.0)

In [6]:
# Predict values to check model performance
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Obtain performance metrics
print("Training set: R2", r2_score(y_train, y_train_pred), "MSE", mean_squared_error(y_train, y_train_pred))
print("Test set: R2", r2_score(y_test, y_test_pred), "MSE", mean_squared_error(y_test, y_test_pred))

Training set: R2 0.9741090435087115 MSE 0.05587297469924072
Test set: R2 0.9754762463968792 MSE 0.052728152646858185


#### Execute jacobian function and check sensitivity metrics

In [7]:
# Obtain parameters to perform jacobian
wts = model.coefs_
bias = model.intercepts_
actfunc = ['identity',model.get_params()['activation'],model.out_activation_]
X = pd.DataFrame(X_train, columns=df.columns[df.columns != 'Y'])
y = pd.DataFrame(y_train, columns=['Y'])
sens_end_layer = 'last'
sens_end_input = False
sens_origin_layer = 0
sens_origin_input = True

In [8]:
jacobian = ns.jacobian_mlp(wts, bias, actfunc, X, y)

In [9]:
ns.alpha_sens_curves(jacobian, max_alpha = 300)